In [6]:
import pandas as pd

In [7]:
import csv

In [8]:
!pip install tweet-preprocessor
# https://github.com/s/preprocessor


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
#before covid
data = pd.read_csv('raw_partner_headlines.csv')

In [ ]:
data.head(5)

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [23]:
data = data.loc[(data['date'] <= '2019-12-31') & (data['date'] >= '2019-01-01')]

import re
from sklearn import feature_extraction 
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

def preprocess(text):
  text = text.lower() #lowercase
  text = re.sub(r'[^\w\s]', '', text) #remove punctuations
  text = re.sub(r'\d+', '', text) #remove numbers
  text = " ".join(text.split()) #stripWhitespace
  text = text.split()
  text = [x for x in text if x not in stop_words] #remove stopwords
  text = [x for x in text if x not in ["stock", "stocks","q", "buy","market","buys"]] #remove task specific stopwords
  text = " ".join(text)
  # stemmer_ps = PorterStemmer()  
  # text = [stemmer_ps.stem(word) for word in text.split()] #stemming
  # text = " ".join(text)
  # lemmatizer = WordNetLemmatizer()
  # text = [lemmatizer.lemmatize(word) for word in text.split()]  #lemmatization
  # text = " ".join(text)
  return(text)


data['headline_processed']=data['headline'].apply(lambda x:preprocess(str(x)))
data['headline_processed']=data['headline_processed'].apply(lambda x:x.split())

from gensim import corpora
dictionary = corpora.Dictionary(data['headline_processed'])
dictionary.filter_extremes(no_below = 100, keep_tokens=['medicine','medical','increase','decrease'])
data['headline_ids']=data['headline_processed'].apply(lambda x:dictionary.doc2bow(x))
from gensim import models
num_topics=10
ldamodel = models.ldamodel.LdaModel(data['headline_ids'], num_topics = num_topics, id2word=dictionary, passes=5, random_state=100)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [45]:
topics = ldamodel.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.074*"dividend" + 0.030*"declares" + 0.021*"bond" + 0.021*"yield" + 0.019*"investors" + 0.017*"high" + 0.016*"march" + 0.016*"class" + 0.016*"action" + 0.016*"reminds"')
(1, '0.125*"earnings" + 0.050*"estimates" + 0.038*"beat" + 0.035*"etfs" + 0.032*"sales" + 0.027*"retail" + 0.025*"revenues" + 0.021*"outlook" + 0.018*"preview" + 0.018*"expected"')
(2, '0.077*"beats" + 0.072*"revenue" + 0.043*"misses" + 0.034*"eps" + 0.024*"day" + 0.019*"conference" + 0.019*"lower" + 0.019*"systems" + 0.019*"economic" + 0.019*"bull"')
(3, '0.062*"watch" + 0.038*"fund" + 0.029*"podcast" + 0.027*"best" + 0.027*"big" + 0.024*"shares" + 0.019*"natural" + 0.017*"rates" + 0.015*"momentum" + 0.015*"likely"')
(4, '0.043*"zumiez" + 0.035*"energy" + 0.028*"gold" + 0.022*"gainers" + 0.020*"volatility" + 0.020*"losers" + 0.020*"tech" + 0.018*"sector" + 0.018*"consumer" + 0.016*"silver"')
(5, '0.052*"portfolio" + 0.049*"update" + 0.049*"trade" + 0.039*"growth" + 0.038*"value" + 0.025*"investors" + 0.019*"righ

In [16]:
from gensim import models
ldamodel.save('model_beforeCovid.lda')

In [51]:
# covid-stage1
data = pd.read_csv('raw_partner_headlines.csv')
data = data.loc[(data['date'] >= '2020-01-01') & (data['date'] <= '2020-03-31')]

import re
from sklearn import feature_extraction 
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

def preprocess(text):
  text = text.lower() #lowercase
  text = re.sub(r'[^\w\s]', '', text) #remove punctuations
  text = re.sub(r'\d+', '', text) #remove numbers
  text = " ".join(text.split()) #stripWhitespace
  text = text.split()
  text = [x for x in text if x not in stop_words] #remove stopwords
  text = [x for x in text if x not in ["stock", "stocks","q", "buy","market","buys","llc","corp","quarter","report","sp","spdr","year","yy","lp","etfs","new"]] #remove task specific stopwords
  text = " ".join(text)
  # stemmer_ps = PorterStemmer()  
  # text = [stemmer_ps.stem(word) for word in text.split()] #stemming
  # text = " ".join(text)
  # lemmatizer = WordNetLemmatizer()
  # text = [lemmatizer.lemmatize(word) for word in text.split()]  #lemmatization
  # text = " ".join(text)
  return(text)


data['headline_processed']=data['headline'].apply(lambda x:preprocess(str(x)))
data['headline_processed']=data['headline_processed'].apply(lambda x:x.split())

from gensim import corpora
dictionary = corpora.Dictionary(data['headline_processed'])
dictionary.filter_extremes(no_below = 100, keep_tokens=['medicine','medical','increase','decrease'])
data['headline_ids']=data['headline_processed'].apply(lambda x:dictionary.doc2bow(x))
from gensim import models
num_topics=20
ldamodel_earlystage = models.ldamodel.LdaModel(data['headline_ids'], num_topics = num_topics, id2word=dictionary, passes=5, random_state=100)

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [53]:
topics = ldamodel_earlystage.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.118*"rating" + 0.054*"watch" + 0.053*"key" + 0.051*"high" + 0.050*"rs" + 0.050*"rise" + 0.047*"hits" + 0.039*"technical" + 0.037*"ibd" + 0.036*"trading"')
(1, '0.136*"global" + 0.132*"growth" + 0.093*"size" + 0.086*"share" + 0.072*"industry" + 0.043*"analysis" + 0.041*"trends" + 0.034*"business" + 0.026*"research" + 0.025*"key"')
(2, '0.120*"review" + 0.101*"equity" + 0.085*"players" + 0.077*"hold" + 0.061*"tiffany" + 0.057*"g" + 0.057*"pdf" + 0.043*"outbreak" + 0.042*"solar" + 0.040*"picks"')
(3, '0.185*"announces" + 0.099*"dow" + 0.069*"jones" + 0.068*"companies" + 0.057*"industrial" + 0.050*"shareholders" + 0.034*"filing" + 0.034*"deal" + 0.031*"limited" + 0.030*"news"')
(4, '0.150*"ishares" + 0.105*"etf" + 0.074*"msci" + 0.068*"shares" + 0.060*"core" + 0.042*"ceo" + 0.040*"sold" + 0.034*"emerging" + 0.034*"markets" + 0.029*"russell"')
(5, '0.131*"partners" + 0.096*"health" + 0.094*"gold" + 0.059*"care" + 0.053*"higher" + 0.049*"heres" + 0.043*"brands" + 0.040*"solutions" + 0

In [27]:
ldamodel_earlystage.save('model_earlystage.lda')

In [48]:
# covid-stage2
data = pd.read_csv('raw_partner_headlines.csv')
data = data.loc[(data['date'] > '2020-03-31') & (data['date'] <= '2020-06-31')]

import re
from sklearn import feature_extraction 
stop_words = feature_extraction.text.ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import nltk 
nltk.download('words')
words = set(nltk.corpus.words.words())

import preprocessor as p
# p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION,p.OPT.HASHTAG)
p.set_options(p.OPT.URL, p.OPT.EMOJI,p.OPT.MENTION)

def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def preprocess(text):
  text = text.lower() #lowercase
  text = p.clean(text)
  text = remove_html_tags(text)
  # text = " ".join(w for w in nltk.wordpunct_tokenize(text) if w.lower() in words or not w.isalpha()) #non-English
  text = re.sub(r'[^\w\s]', '', text) #remove punctuations
  text = re.sub(r'\d+', '', text) #remove numbers
  text = " ".join(text.split()) #stripWhitespace
  text = text.split()
  text = [x for x in text if x not in stop_words] #remove stopwords
  text = [x for x in text if x not in ['stock','stocks','covid','report','corp','llc','ishares','etf','spdr','sf','msci','amid']] #remove task specific stopwords
  text = [x for x in text if len(x)>2]
  text = " ".join(text)
  # stemmer_ps = PorterStemmer()  
  # text = [stemmer_ps.stem(word) for word in text.split()] #stemming
  # text = " ".join(text)
  # lemmatizer = WordNetLemmatizer()
  # text = [lemmatizer.lemmatize(word) for word in text.split()]  #lemmatization
  # text = " ".join(text)
  return(text)


data['headline_processed']=data['headline'].apply(lambda x:preprocess(str(x)))
data['headline_processed']=data['headline_processed'].apply(lambda x:x.split())

from gensim import corpora
dictionary = corpora.Dictionary(data['headline_processed'])
dictionary.filter_extremes(no_below = 100, keep_tokens=['medicine','medical','increase','decrease'])
data['headline_ids']=data['headline_processed'].apply(lambda x:dictionary.doc2bow(x))
from gensim import models
num_topics=20
ldamodel_third = models.ldamodel.LdaModel(data['headline_ids'], num_topics = num_topics, id2word=dictionary, passes=5, random_state=100)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [52]:
topics = ldamodel_third.print_topics(num_words=10)
for i in range(num_topics):
    print(topics[i])

(0, '0.151*"international" + 0.062*"associates" + 0.057*"brands" + 0.048*"air" + 0.045*"proshares" + 0.044*"class" + 0.044*"medical" + 0.043*"technology" + 0.041*"products" + 0.038*"trading"')
(1, '0.083*"strength" + 0.067*"relative" + 0.066*"rating" + 0.050*"united" + 0.046*"highlights" + 0.044*"zacks" + 0.042*"rising" + 0.040*"silver" + 0.039*"price" + 0.038*"analyst"')
(2, '0.121*"shares" + 0.086*"dividend" + 0.074*"sold" + 0.073*"midcap" + 0.071*"million" + 0.065*"ceo" + 0.043*"point" + 0.040*"president" + 0.037*"pharmaceuticals" + 0.035*"royal"')
(3, '0.267*"market" + 0.118*"global" + 0.090*"april" + 0.088*"growth" + 0.051*"thursday" + 0.039*"disney" + 0.038*"walt" + 0.034*"key" + 0.032*"monday" + 0.030*"equity"')
(4, '0.149*"sector" + 0.122*"select" + 0.113*"fund" + 0.095*"buys" + 0.077*"services" + 0.042*"advisory" + 0.040*"financial" + 0.031*"edge" + 0.026*"usa" + 0.018*"research"')
(5, '0.310*"earnings" + 0.191*"estimates" + 0.086*"revenues" + 0.078*"beat" + 0.046*"sales" + 0.

In [40]:
ldamodel_third.save('model_secondstage.lda')